In [ ]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
from PyISR import ISRSolverSLE
from ROOT import TFile
import matplotlib
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.sans-serif'] = ['Tahoma', 'DejaVu Sans',
                                          'Lucida Grande', 'Verdana']

In [ ]:
from tqdm import tqdm

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline

In [ ]:
def readVCS(path):
    fl = TFile.Open(path, "read")
    # Getting a pointer to the visible cross section in the form of TGraphErrors
    gvcs = fl.Get('vcs')
    # Number of c.m. energy points
    n = gvcs.GetN()
    # Reading c.m. energy array
    energy = np.frombuffer(gvcs.GetX(), dtype=np.float64, count=n)
    # Reading array of c.m. energy errors
    energyErr = np.frombuffer(gvcs.GetEX(), dtype=np.float64, count=n)
    # Reading visible cross section array
    vcs = np.frombuffer(gvcs.GetY(), dtype=np.float64, count=n)
    # Reading array of visible cross section errors
    vcsErr = np.frombuffer(gvcs.GetEY(), dtype=np.float64, count=n)
    fl.Close()
    return energy, vcs, energyErr, vcsErr

In [ ]:
def solve(energy, vcs, energyErr, vcsErr, 
          enabled_energy_spread=False, 
          threshold_energy=0.827, 
          interp=None,
          efficiency=lambda x, en: 1.0):
    n = energy.shape[0]
    solver = ISRSolverSLE(n, energy, vcs, 
                          energyErr, vcsErr, 
                          threshold_energy, efficiency,
                          enabled_energy_spread)
    if type(interp) == str:
        with open(interp, 'r') as jfl:
            settings = json.load(jfl)
        
        print('Interpolation settings:')
        print(settings)
        solver.set_interp_settings(settings)
    elif type(interp) == list:
        print('Interpolation settings:')
        print(interp)
        solver.set_interp_settings(interp)
    
    solver.solve()
    return solver

In [ ]:
input_path = 'data/gen_visible_cs_etapipi_simple_model_no_energy_spread.root'
solver = solve(*readVCS(input_path))
enspread = np.linspace(0.,0.014, 100)
condnum = [solver.condnum_eval(),]
solver.energy_spread_enabled = True
pbar = tqdm(total=enspread.shape[0] -1)
for i in range(1, enspread.shape[0]):
    solver.reset_ecm_err(enspread[i] * np.ones(solver.n))
    solver.eval_equation_matrix()
    condnum.append(solver.condnum_eval())
    pbar.update(1)
    
condnum = np.array(condnum)

In [ ]:
def condnumPlot(x, y):
    fontsize = 24
    f, ax = plt.subplots(figsize=(9, 7))
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    ax.plot(x * 1.e+3, y, 'r-')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\sigma_E$ (MeV)', fontsize=fontsize)
    ax.set_ylabel('condition number', fontsize=fontsize)

In [ ]:
condnumPlot(enspread, condnum)